In [91]:
Today = '31/03/24'
Mounth = 'mar'

print_Debt = True
print_Deriv = True
print_BD = True

In [2]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

import pandas as pd
from datetime import date

In [3]:
query = f"""select * from "RISKACCESS"."quantumDebt" where "reportDate" = TO_DATE('{Today}', 'DD/MM/YY')"""
data_Debt_export = export_from_RISKCUSTOM(query)
data_Debt_export.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,##batch_id,##deleted,##inserted,##origin_name,flag,actionDate,amount,eventCurrency,amountUSD,currentRate,...,subsidised,debtRelated,reportDate,eventNumber,entitySAPBUCode,counterpartySAPID,counterpartySWIFTBICCode,effectiveFrom,xlsDate,loadEventId
0,35133836@WHPROD,N,2024-04-01 23:35:25,QUANTUM,-,2038-04-26,-232364.04,RUB,-2515.688024,3.0,...,YES,Y,2024-03-31,198823,VG00,1000147688,None,2024-04-01,2024-04-01 23:34:06,96881
1,35133836@WHPROD,N,2024-04-01 23:35:25,QUANTUM,-,2038-07-26,-233293.87,RUB,-2525.754823,3.0,...,YES,Y,2024-03-31,198824,VG00,1000147688,None,2024-04-01,2024-04-01 23:34:06,96881


In [9]:
data_Debt_export.holding.unique()

array(['SUEK', 'EuroChem'], dtype=object)

Фильтрация данных

In [4]:
max_data = data_Debt_export.effectiveFrom.max() # Присваивание переменной значения мин даты в столбце effectiveFrom

for i in ['SUEK', 'EuroChem']:
    data_Debt_work = data_Debt_export.query('holding == @i & effectiveFrom == @max_data').reset_index(drop=True) # Фильтрация данных по столбцу holding (переменная Group) и effectiveFrom
    data_Debt_work = data_Debt_work[~data_Debt_work.entity.isna()]
    data_Debt_work.entity = data_Debt_work.entity.replace({'RUBND': 'RUSBO'}).reset_index(drop=True)
    if i == 'SUEK':
        data_Debt_work = data_Debt_work.rename(columns={'entity': 'pre_entity'})
        data_Debt_work['entity'] = merge_Mapping(data_Debt_work, col='pre_entity')

    data_Debt_work['Entity_group'] = merge_SalesUnits(data_Debt_work, col='entity', merge_col='ocpSegment').fillna('External')

    # Выполнение цикла для рассчета amount, amountUSD и присваивания значения counterparty для каждого DealSet

    deals_data = data_Debt_work[['entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']] # Выбор нужного разреза данных
    deals_data_Bonds = deals_data[deals_data.dealSet.str.contains('Bond')] # Фильтрация разреза по значениям в DealSet, которые содержат слово Bond

    Bonds_list = deals_data_Bonds.dealSet.unique()

    for Bond in Bonds_list:
        try:
            Bond_data = deals_data_Bonds.query('dealSet ==@Bond') # Отбор данных одного Bond
            assert len(Bond_data[Bond_data.instrumentOwner == 'EC_ISSUED_BONDS']) != 0

            for column in Bond_data[['amount', 'amountUSD']]:
                Traded_data_sum = Bond_data.query('instrumentOwner =="EC_TRADED_BONDS"')[column].sum()
                Issued_data = Bond_data.query('instrumentOwner =="EC_ISSUED_BONDS"')[column]
                # Внесение нового значения amount в issued bonds
                Bond_data.loc[Bond_data['instrumentOwner'] =="EC_ISSUED_BONDS", column] = Traded_data_sum + Issued_data 
            # Внесение нового значения counterparty в traded bonds
            Issied_data_entity = Bond_data.query('instrumentOwner =="EC_ISSUED_BONDS"').Entity.squeeze()
            Bond_data.loc[Bond_data['instrumentOwner'] =="EC_TRADED_BONDS", 'counterparty'] = Issied_data_entity
            deals_data_Bonds.loc[deals_data_Bonds['dealSet'] == Bond] = Bond_data
        except:
            continue

    deals_data.loc[deals_data.dealSet.str.contains('Bond')] = deals_data_Bonds
    data_Debt_work[['Entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']] = deals_data
    # Запись выполненного цикла в в основные данные

    data_Debt_work = Period(data_Debt_work, day_for_count=Today, col_with_date='actionDate').reset_index(drop=True)

    data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'] = data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5] 
    # Обрезка " (EC)" в каждой строке

    data_Debt_work = data_Debt_work[~data_Debt_work.counterparty.isna()].reset_index(drop=True)
    data_Debt_work['CompCode'] = merge_Mapping(data_Debt_work, col='counterparty').fillna('External')
    data_Debt_work['Counterparty_Group'] = merge_SalesUnits(data_Debt_work, col='CompCode', merge_col='ocpSegment').fillna('External')
    # Merge counterparty с Mapping и SalesUnits

    data_Debt_work['Source'] = 'Quantum'

    if i == 'SUEK':
        data_Debt_work_SUEK = data_Debt_work
    if i == 'EuroChem':
        data_Debt_work_ECH = data_Debt_work

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9204\3394882411.py:43: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'] = data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5]
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:80: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql

Запись Debt в Excel:

In [14]:
# Sheet_in_output_Deriv = 'Deriv'
# Output_Deriv = "_".join([str(date.today()), Group, 'quantum', 'Deriv', f'{Mounth}.xlsx'])

Sheet_in_output_Debt = 'Debt'
Output_Debt_SUEK = "_".join([str(date.today()), 'SUEK_quantum_Debt', f'{Mounth}.xlsx'])
Output_Debt_ECH = "_".join([str(date.today()), 'Ech_quantum_Debt', f'{Mounth}.xlsx'])

data_Debt_export_for_print_SUEK = data_Debt_work_SUEK[['entity', 'Entity_group','counterparty',\
                                'Counterparty_Group','amount','eventCurrency','amountUSD','actionDate','Days',\
                                'Period','dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]\
                                .rename(columns={'amount': 'amoutn outstanding',\
                                                    'eventCurrency': 'Currency',\
                                                    'amountUSD': 'amount USD eq',\
                                                    'actionDate': 'termEnd'})
data_Debt_export_for_print_ECH = data_Debt_work_ECH[['entity', 'Entity_group','counterparty',\
                                'Counterparty_Group','amount','eventCurrency','amountUSD','actionDate','Days',\
                                'Period','dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]\
                                .rename(columns={'amount': 'amoutn outstanding',\
                                                    'eventCurrency': 'Currency',\
                                                    'amountUSD': 'amount USD eq',\
                                                    'actionDate': 'termEnd'})

if print_Debt == True:
    data_Debt_export_for_print_SUEK.to_excel(Output_Debt_SUEK, sheet_name = Sheet_in_output_Debt, index = False)
    data_Debt_export_for_print_ECH.to_excel(Output_Debt_ECH, sheet_name = Sheet_in_output_Debt, index = False) 

In [12]:
check = data_Debt_work[['Entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']]
check = check[check.instrumentOwner.str.contains('BONDS')].reset_index(drop=True)
check[check.dealSet == check.dealSet.unique().tolist()[3]]

,Entity,Entity_group,counterparty,amount,amountUSD,instrumentOwner,dealSet
74,1100,SUEK RU,1800,200000.0,200000.0,EC_TRADED_BONDS,SUEK RuBond 07 2023 $300m B
75,1100,SUEK RU,1800,200000.0,200000.0,EC_TRADED_BONDS,SUEK RuBond 07 2023 $300m B
86,1100,SUEK RU,1800,350000.0,350000.0,EC_TRADED_BONDS,SUEK RuBond 07 2023 $300m B
87,1100,SUEK RU,1800,14950000.0,14950000.0,EC_TRADED_BONDS,SUEK RuBond 07 2023 $300m B
105,1800,SUEK RU,NSD RU (EC),-284300000.0,-284300000.0,EC_ISSUED_BONDS,SUEK RuBond 07 2023 $300m B


In [17]:
manual_map = data_Debt_work.loc[(data_Debt_work['cpartyType'] == 'Internal') & (data_Debt_work['CompCode'] == 'External'), ['counterparty', 'CompCode', 'Counterparty_Group']].drop_duplicates()
manual_map

manual_map2 = data_Debt_work.loc[data_Debt_work['Entity_group'] == 'External', ['entity', 'Entity_group']]
manual_map2

manual_map_print = pd.concat([manual_map, manual_map2], axis=1)
manual_map_print
# manual_map_print.to_excel('Manual_map.xlsx', index=False)

,counterparty,CompCode,Counterparty_Group,entity,Entity_group


In [9]:
# data_Debt_export.to_excel(Output_Debt, sheet_name = Sheet_in_output_Debt, index = False)
# Полные данные для проверки качества выполнения кода

Работа с Forwards и Swaps:

In [16]:
data_Forwards_export = pd.read_excel(Data_Forwards, sheet_name = Sheet_in_data_file)
data_Swaps_export = pd.read_excel(Data_Swaps, sheet_name = Sheet_in_data_file)

c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [17]:
data_Forwards_export['holdingEntity'] = data_Forwards_export['holdingEntity'].replace({'XXTSU': 'SUEK'})
data_Forwards_export = data_Forwards_export[data_Forwards_export['holdingEntity'] == Group].reset_index(drop=True)
data_Swaps_export['holdingEntity'] = data_Swaps_export['holdingEntity'].replace({'XXTSU': 'SUEK'})
data_Swaps_export = data_Swaps_export[data_Swaps_export['holdingEntity'] == Group].reset_index(drop=True)
# Знамена значений в стодбце holdingEntity и фильтр по Group

Проверка вхождений dealNo в Debt и остановка процесса в случае отсутвия данных:

In [18]:
Swap_dealNo_for_del_list = list(set(data_Swaps_export['dealNo']).intersection(set(data_Debt_export['dealNo'])))
Forward_dealNo_for_del_list = list(set(data_Forwards_export['dealNo']).intersection(set(data_Debt_export['dealNo'])))

data_Swaps_export = data_Swaps_export[~(data_Swaps_export.dealNo.isin(Swap_dealNo_for_del_list))]
data_Forwards_export = data_Forwards_export[~(data_Forwards_export.dealNo.isin(Forward_dealNo_for_del_list))]

assert ((len(data_Swaps_export) == 0) and (len(data_Forwards_export) == 0)) == False

AssertionError: 

In [55]:
Data_for_loop = data_Forwards_export[['entity','holdingEntity','counterparty','dealNo', 'payFaceValue', 'payFXCurrency', 'recFaceValue', 'recFXCurrency', 'maturityDate']]

for i in range(0, len(Data_for_loop)):
    row_list = list(Data_for_loop.loc[i])
    Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
    Data_for_loop.loc[i,'payFaceValue'] = Data_for_loop.loc[i,'payFaceValue'] * -1
Data_for_loop = Data_for_loop.sort_values('dealNo')
# цикл создания новых строк

Data_for_loop['Entity_group'] = merge_SalesUnits(Data_for_loop, col='entity', merge_col='ocpSegment').fillna('External')
# Merge столбца entity по mapping

Data_for_loop = Period(Data_for_loop, col_with_date='maturityDate', day_for_count=Today)
# Создание Days и Period

Data_for_loop['USD_equiv'] = 0
for i in range (0,len(Data_for_loop)):
    if Data_for_loop.loc[i, 'payFXCurrency'] == 'CNY':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * CNY_USD
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'RUB':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * RUB_USD
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'USD':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue']
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'CAD':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * CAD_USD
# создание USD_equiv

new_columns = ['Company', 'Settlement', 'Notional Amount (USD)', 'Unnamed', 'source']
Data_for_loop = Data_for_loop.reindex(columns=(Data_for_loop.columns.tolist() + new_columns))
Data_for_loop['source'] = 'Quantum'
# Создание пустых столбцов и столбца ресурса

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9520\659122996.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9520\659122996.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9520\659122996.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

In [56]:
Data_for_loop_Swap = data_Swaps_export[['entity','holdingEntity','counterparty','dealNo', 'payFaceValue', 'payFXCurrency', 'recFaceValue', 'recFXCurrency', 'maturityDate']]

for i in range(0, len(Data_for_loop_Swap)):
    row_list = list(Data_for_loop_Swap.loc[i])
    Data_for_loop_Swap.loc[len(Data_for_loop_Swap)] = row_list[:4] + row_list[6:8] + row_list[6:]
    Data_for_loop_Swap.loc[i,'payFaceValue'] = Data_for_loop_Swap.loc[i,'payFaceValue'] * -1
Data_for_loop_Swap = Data_for_loop_Swap.sort_values('dealNo')
# цикл создания новых строк

Data_for_loop_Swap['Entity_group'] = merge_SalesUnits(Data_for_loop_Swap, col='entity', merge_col='ocpSegment').fillna('External')
# Merge столбца entity по mapping

Data_for_loop_Swap = Period(Data_for_loop_Swap, day_for_count=Today, col_with_date='maturityDate')
# Создание Days и Period

Data_for_loop_Swap['USD_equiv'] = 0
for i in range (0,len(Data_for_loop_Swap)):
    if Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'CNY':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue'] * CNY_USD
    elif Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'RUB':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue'] * RUB_USD
    elif Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'USD':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue']
# создание USD_equiv

new_columns = ['Company', 'Settlement', 'Notional Amount (USD)', 'Unnamed', 'source']
Data_for_loop_Swap = Data_for_loop_Swap.reindex(columns=(Data_for_loop_Swap.columns.tolist() + new_columns))
Data_for_loop_Swap['source'] = 'Quantum'
# Создание пустых столбцов и столбца ресурса

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:44: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  


In [53]:
data = Data_for_loop[['entity','Entity_group','holdingEntity','Company',\
                    'counterparty','dealNo','payFXCurrency','payFaceValue',\
                    'USD_equiv','Settlement','Notional Amount (USD)',\
                    'Unnamed','maturityDate','Days','Period','source']]

data_Swap = Data_for_loop_Swap[['entity','Entity_group','holdingEntity','Company',\
                    'counterparty','dealNo','payFXCurrency','payFaceValue',\
                    'USD_equiv','Settlement','Notional Amount (USD)',\
                    'Unnamed','maturityDate','Days','Period','source']]
# Выбор столбцов

data_to_excel = pd.concat([data, data_Swap], axis=0)
# Объединение таблиц вниз

In [57]:
data_to_excel.loc[data_to_excel['Entity_group'] == 'External']

,entity,Entity_group,holdingEntity,Company,counterparty,dealNo,payFXCurrency,payFaceValue,USD_equiv,Settlement,Notional Amount (USD),Unnamed,maturityDate,Days,Period,source


In [ ]:
data_to_excel.to_excel(Output_Deriv, sheet_name = Sheet_in_output_Deriv, index = False)
# Запись в файл 

Тестовая БД:

In [117]:
query = """
select * from "unifiedOcp"
where "sourceSystem" in ('QUANTUM', 'BANK_ACCOUNTS_BALANCE_XLS')
and "reportDate" = TO_DATE('2024-03-31', 'YYYY-MM-DD')
"""

data = export_from_WHWEEK(query)

data_work = data.query('holding == "EUROCHEM"')[['##origin_name', 'holding', 'buCode','ocpSegment', 'counterpartyName', 'effectiveCurrency', 'volumeEffectiveCurrency', 'maturityDateToDays','timeBucket']]
data_work.loc[data_work.counterpartyName.str.contains('(EC)'), 'counterpartyName'] = data_work.loc[data_work.counterpartyName.str.contains('(EC)'), 'counterpartyName'].str[:-5] # Обрезка " (EC)" в каждой строке
data_work = concat_columns(df=data_work, columns=['buCode', 'counterpartyName', 'effectiveCurrency', 'maturityDateToDays',])

data_test = data_Debt_work_ECH[['##origin_name', 'holding', 'entity', 'Entity_group','counterparty',\
                                'eventCurrency','amount', 'Days',\
                                'Period']]
data_test = concat_columns(data_test, ['entity', 'counterparty', 'eventCurrency', 'Days'])
 
equal_list = [i for i in data_test.concat_columns.tolist() if i in data_work.concat_columns.tolist()]
data_merge = data_test[data_test.concat_columns.isin(equal_list)].reset_index(drop=True).merge(data_work.reset_index(drop=True), how='left', left_on='concat_columns', right_on='concat_columns')
data_merge['Amount_is_equal'] = data_merge.amount.abs() == data_merge.volumeEffectiveCurrency.abs()
data_merge['index_copy'] = data_merge.index

def accident_count(group):
    c = group['index_copy'].count()
    group['num_records'] = c

    return group

data_merge_groupby = data_merge.groupby(['##origin_name_x',	'holding_x',	'entity',	'Entity_group',	'counterparty',	'eventCurrency',	'amount',	'Days',	'Period',	'concat_columns']).apply(accident_count)
data_merge_groupby = data_merge_groupby.iloc[:, 10:]

data_merge_groupby

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:159: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9204\2923683032.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_work.loc[data_work.counterpartyName.str.contains('(EC)'), 'counterpartyName'] = data_work.loc[data_work.counterpartyName.str.contains('(EC)'), 'counterpartyName'].str[:-5] # Обрезка " (EC)" в каждой строке
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

##origin_name_y  \
##origin_name_x holding_x entity Entity_group counterparty       eventCurrency amount        Days Period concat_columns                                           
QUANTUM         EuroChem  AECOR  TRADING      Nadati Trading     USD            0.000000e+00 277  6M-1Y  AECOR_Nadati Trading_USD_277      3757          WHFLEX   
                                                                                2.200000e+07 1005 2Y+    AECOR_Nadati Trading_USD_1005     3758          WHFLEX   
                          AETRD  TRADING      Axioma             USD           -6.000000e+07 66   1M-3M  AETRD_Axioma_USD_66               4160          WHFLEX   
                                              Corrigo            USD           -4.000000e+06 1    <1M    AETRD_Corrigo_USD_1               3759          WHFLEX   
                                                                                0.000000e+00 284  6M-1Y  AETRD_Corrigo_USD_284             3760          WHFLEX   
...                                                                                                                                                         ...   
                          RUVGK  RUFERT       Sberbank RU Moscow RUB           -4.431029e+10 1004 2Y+    RUVGK_Sberbank RU Moscow_RUB_1004 4483          WHFLEX   
                                                                                0.000000e+00 284  6M-1Y  RUVGK_Sberbank RU Moscow_RUB_284  4482          WHFLEX   
                                                                                7.912994e+07 2    <1M    RUVGK_Sberbank RU Moscow_RUB_2    4479          WHFLEX   
                                                                                1.342181e+08 11   <1M    RUVGK_Sberbank RU Moscow_RUB_11   4481          WHFLEX   
                                                                                1.957301e+08 5    <1M    RUVGK_Sberbank RU Moscow_RUB_5    4480          WHFLEX   

                                                                                                                                                holding_y  \
##origin_name_x holding_x entity Entity_group counterparty       eventCurrency amount        Days Period concat_columns                                     
QUANTUM         EuroChem  AECOR  TRADING      Nadati Trading     USD            0.000000e+00 277  6M-1Y  AECOR_Nadati Trading_USD_277      3757  EUROCHEM   
                                                                                2.200000e+07 1005 2Y+    AECOR_Nadati Trading_USD_1005     3758  EUROCHEM   
                          AETRD  TRADING      Axioma             USD           -6.000000e+07 66   1M-3M  AETRD_Axioma_USD_66               4160  EUROCHEM   
                                              Corrigo            USD           -4.000000e+06 1    <1M    AETRD_Corrigo_USD_1               3759  EUROCHEM   
                                                                                0.000000e+00 284  6M-1Y  AETRD_Corrigo_USD_284             3760  EUROCHEM   
...                                                                                                                                                   ...   
                          RUVGK  RUFERT       Sberbank RU Moscow RUB           -4.431029e+10 1004 2Y+    RUVGK_Sberbank RU Moscow_RUB_1004 4483  EUROCHEM   
                                                                                0.000000e+00 284  6M-1Y  RUVGK_Sberbank RU Moscow_RUB_284  4482  EUROCHEM   
                                                                                7.912994e+07 2    <1M    RUVGK_Sberbank RU Moscow_RUB_2    4479  EUROCHEM   
                                                                                1.342181e+08 11   <1M    RUVGK_Sberbank RU Moscow_RUB_11   4481  EUROCHEM   
                                                                                1.957301e+08 5    <1M    RUVGK_Sberbank RU Moscow_RUB_5    4

In [128]:
data_QUANTUM_not_equal = data_test[~data_test.concat_columns.isin(equal_list)]
data_WHFLEX_not_equal = data_work[~data_work.concat_columns.isin(equal_list)]
data_WHFLEX_not_equal.columns = data_QUANTUM_not_equal.columns.tolist()

not_equal = pd.concat([data_QUANTUM_not_equal, pd.DataFrame(columns=data_QUANTUM_not_equal.columns, index=[None] * 2), data_WHFLEX_not_equal], ignore_index=True)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9204\2518512979.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  not_equal = pd.concat([data_QUANTUM_not_equal, pd.DataFrame(columns=data_QUANTUM_not_equal.columns, index=[None] * 2), data_WHFLEX_not_equal], ignore_index=True)


In [98]:
Output_BD = "_".join([str(date.today()), 'QUANTUM_to_WHFLEX', f'{Mounth}.xlsx'])
if print_BD == True:
    write_data = {'data': ['Слева-направо Вы ыидите выгрузку из QUANTUM и соответсвюущие ей строки из WHFLEX', '',
                           'Лист equal_records содержит данные о совпавших строках для обоих источников (сопоставление производилось по коду компании, контрагенту, валюте операции и давности операции в днях)',
                           'Столбец num_records подскажет сколько строк WHFLEX соответсвует одной строке QUANTUM',
                           'Поле Amount_is_equal сравнивает объем денежного потока из обоих источников. Если поле равно 1\True, то денежные потоки равны']}
    df = pd.DataFrame(write_data)
    df.to_excel(Output_BD, sheet_name='Iinstruction', index=False, header=False)
    
    new_list(not_equal, Output_BD, sheet_name='Not_equal_records', index=True)
    new_list(data_merge_groupby, Output_BD, sheet_name='Equal_records', index=True)